In [8]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

from src.globals import *

In [9]:
df_imu = pd.read_parquet(path_imu)
df_motor = pd.read_parquet(path_motor)

In [10]:
# Min Max Normalization IMU
for sensor, axes in imu_sensor_axes:
    columns = [f'{sensor}_{axis}' for axis in axes]
    values = df_imu[columns].values

    scaler = MinMaxScaler((-1, 1)).fit(values.reshape(-1, 1))
    for col in columns:
        df_imu[col] = scaler.transform(df_imu[col].values.reshape(-1, 1))

df_motor[motor_sensors_u] = MinMaxScaler((-1, 1)).fit_transform(df_motor[motor_sensors_u])

In [11]:
def cal_lof(df, columns, colname):
    lof = LocalOutlierFactor()
    lof.fit(df[['timeindex_bin'] + columns])
    df[colname] = -lof.negative_outlier_factor_
    return df

# IMU
for sensor,axes in imu_sensor_axes:
    cal_lof(df_imu, [f'{sensor}_{axis}' for axis in axes], f'lof_{sensor}')
cal_lof(df_imu, [f'{sensor}_{axis}' for sensor in ['acceleration', 'gyroscope'] for axis in imu_axes3], 'lof_acc_gyro')


# Motor
cal_lof(df_motor, motor_sensors, 'lof_motor')

,timeindex_bin,seqid,timeindex,angle_0,angle_1,angle_2,output_0,output_1,output_2,label,anomaly,lof_motor
0,0,2024.09.16|00.01.03,0.004796,0.022231,0.019642,0.021846,-0.040323,-0.182874,-0.061580,run0,none,1.745431
1,1,2024.09.16|00.01.03,0.024186,0.021785,0.019248,0.021407,-0.041475,-0.180697,-0.062584,run0,none,1.218224
2,2,2024.09.16|00.01.03,0.043576,0.021338,0.018854,0.020969,-0.042627,-0.178520,-0.063588,run0,none,1.021175
3,3,2024.09.16|00.01.03,0.062966,0.020892,0.018459,0.020530,-0.043779,-0.176343,-0.064592,run0,none,0.975891
4,4,2024.09.16|00.01.03,0.082356,0.020445,0.018065,0.020091,-0.044931,-0.174165,-0.065596,run0,none,1.199154
...,...,...,...,...,...,...,...,...,...,...,...,...
1489344,4129,2024.09.16|23.59.09,87.709658,3.777704,3.802446,3.796205,-0.008065,-0.142235,-0.033467,run012,none,1.271985
1489345,4130,2024.09.16|23.59.09,87.731134,3.715240,3.739492,3.733375,-0.011137,-0.144170,-0.035252,run012,none,1.496007
1489346,4131,2024.09.16|23.59.09,87.752610,3.652776,3.676538,3.670544,-0.014209,-0.146105,-0.037037,run012,none,1.660509
1489347,4132,2024.09.16|23.59.09,87.774086,3.590312,3.613584,3.607713,-0.017281,-0.148041,-0.038822,run012,none,1.300955


# LOF results to results database

In [20]:
df_imu_results = pd.read_parquet(path_imu_results)
df_motor_results = pd.read_parquet(path_motor_results)

lofcols = ['seqid', 'timeindex_bin'] + [f'lof_{sensor}' for sensor in imu_sensors] + ['lof_acc_gyro']
df_imu_results = df_imu_results.merge(df_imu[lofcols], on=['seqid', 'timeindex_bin'], suffixes=('', '_drop'))
df_imu_results = df_imu_results.drop([col for col in df_imu_results.columns if col.endswith('_drop')], axis=1)

df_motor_results = df_motor_results.merge(df_motor[['seqid', 'timeindex_bin', 'lof_motor']], on=['seqid', 'timeindex_bin'], suffixes=('', '_drop'))
df_motor_results = df_motor_results.drop([col for col in df_motor_results.columns if col.endswith('_drop')], axis=1)

df_imu_results.to_parquet(path_imu_results)
df_motor_results.to_parquet(path_motor_results)

In [27]:
# Creating the series results
func = lambda x : np.sum(abs(x) >= threshold_lof)

df_series_imu = df_imu.groupby(['seqid']).agg({f'lof_{sensor}' : func for sensor in imu_sensors} | {'lof_acc_gyro' : func}
                                           ).reset_index()

df_series_motor = df_motor.groupby(['seqid']).agg({'lof_motor' : func}).reset_index()

df_series = df_series_imu.merge(df_series_motor, on='seqid')


In [29]:
df_series_results = pd.read_parquet(path_series_results)

lofcols = [f'lof_{sensor}' for sensor in imu_sensors]
df_series_results = df_series_results.merge(df_series[['seqid', 'lof_acc_gyro', 'lof_motor'] + lofcols], on=['seqid'], suffixes=('', '_drop'))
df_series_results = df_series_results.drop([col for col in df_series_results.columns if col.endswith('_drop')], axis=1)

df_series_results.to_parquet(path_series_results)

,seqid,anomaly,lof_all,lof_acceleration,lof_gravity,lof_gyroscope,lof_magnetic,lof_rotation,lof_acc_gyro,lof_motor
0,2024.09.16|00.01.03,none,41,111,27,162,8,6,110,548
1,2024.09.16|00.02.37,none,52,159,216,177,24,19,115,587
2,2024.09.16|00.04.10,none,34,158,103,124,65,6,105,660
3,2024.09.16|00.05.43,none,30,126,81,180,17,5,103,545
4,2024.09.16|00.07.16,none,47,196,85,236,17,108,172,545
...,...,...,...,...,...,...,...,...,...,...
356,2024.09.16|23.53.08,none,20,122,45,131,21,3,69,505
357,2024.09.16|23.54.38,none,50,245,98,211,62,103,136,527
358,2024.09.16|23.56.09,none,26,145,69,187,26,2,80,529
359,2024.09.16|23.57.39,none,22,110,59,163,17,4,93,476
